In [18]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize
from scipy.stats import norm

In [19]:
data_df = pd.read_csv('data/Project-Data.csv')

In [20]:
data_df.head()

,Maturity,ZCYield,FuturesPrice
0,0.002778,0.014498,79.998782
1,0.020833,0.014426,79.994830
2,0.083333,0.014444,79.973349
3,0.166667,0.014399,79.954710
4,0.250000,0.014336,79.911934


## Bond pricing

In [21]:
sigma_r = 0.00023

maturities = data_df['Maturity'].values
zc_yields = data_df['ZCYield'].values

In [22]:
def b0(T, t, lambda_r, sigma_r, r_bar):
    return (np.exp(-2 * lambda_r * (T-t)) * (np.exp(lambda_r * (T-t)) - 1) *
            (sigma_r**2 + np.exp(lambda_r * (T-t)) * (4 * lambda_r**2 * r_bar - 3 * sigma_r**2)) +
            2 * lambda_r * (T-t) * (sigma_r**2 - 2 * lambda_r**2 * r_bar)) / (4 * lambda_r**3)

def br(T, t, lambda_r):
    return (np.exp(-lambda_r * (T-t)) - 1) / lambda_r

In [23]:
initial_guess = [0.1, 0.03, 0.01]

### Least Squares Calibration

In [30]:
def least_squares_objective(params):
    lambda_r, r_bar, r0 = params
    error = 0
    for maturity, yield_ in zip(maturities, zc_yields):
        T = maturity
        t = 0  # Assuming current time is 0
        P_model = np.exp(b0(T, t, lambda_r, sigma_r, r_bar) + br(T, t, lambda_r) * r0)
        P_market = np.exp(-yield_ * (T-t))
        error += (P_model - P_market) ** 2
    return error

least_squares_result = minimize(least_squares_objective, initial_guess, method='Nelder-Mead')
lambda_r_ls, r_bar_ls, r0_ls = least_squares_result.x
print(f'Least Squares Calibration:')
print(f'Mean reversion intensity (lambda_r): {lambda_r_ls}')
print(f'Long-term mean (r_bar): {r_bar_ls}')
print(f'Initial value (r0): {r0_ls}')

Least Squares Calibration:
Mean reversion intensity (lambda_r): 0.2991437388948194
Long-term mean (r_bar): 0.010033839633324947
Initial value (r0): 0.014603967286584005


### Maximum Likelihood Calibration

In [26]:
def log_likelihood(params):
    lambda_r, r_bar, r0 = params
    likelihood = 0
    for maturity, yield_ in zip(maturities, zc_yields):
        T = maturity
        t = 0  # Assuming current time is 0
        P_model = np.exp(b0(T, t, lambda_r, sigma_r, r_bar) + br(T, t, lambda_r) * r0)
        P_market = np.exp(-yield_ * (T-t))
        likelihood += norm.logpdf(P_market, loc=P_model, scale=sigma_r)
    return -likelihood  # Negative log-likelihood

mle_result = minimize(log_likelihood, initial_guess, method='Nelder-Mead')
lambda_r_mle, r_bar_mle, r0_mle = mle_result.x
print(f'Maximum Likelihood Calibration:')
print(f'Mean reversion intensity (lambda_r): {lambda_r_mle}')
print(f'Long-term mean (r_bar): {r_bar_mle}')
print(f'Initial value (r0): {r0_mle}')

Maximum Likelihood Calibration:
Mean reversion intensity (lambda_r): 0.2991437388948194
Long-term mean (r_bar): 0.010033839633324947
Initial value (r0): 0.014603967286584005


### Long Term Quantile Method

In [27]:
def quantile_objective(params):
    lambda_r, r_bar, r0 = params
    quantile_error = 0
    for maturity, yield_ in zip(maturities, zc_yields):
        T = maturity
        t = 0  # Assuming current time is 0
        P_model = np.exp(b0(T, t, lambda_r, sigma_r, r_bar) + br(T, t, lambda_r) * r0)
        P_market = np.exp(-yield_ * (T-t))
        quantile_error += np.abs(P_model - P_market)  # Absolute error for simplicity
    return quantile_error

# Perform the optimization
quantile_result = minimize(quantile_objective, initial_guess, method='Nelder-Mead')

# Extract the calibrated parameters
lambda_r_quantile, r_bar_quantile, r0_quantile = quantile_result.x

print(f'Long Term Quantile Calibration:')
print(f'Mean reversion intensity (lambda_r): {lambda_r_quantile}')
print(f'Long-term mean (r_bar): {r_bar_quantile}')
print(f'Initial value (r0): {r0_quantile}')

Long Term Quantile Calibration:
Mean reversion intensity (lambda_r): 0.16322924742036396
Long-term mean (r_bar): 0.008550775735612639
Initial value (r0): 0.01429913713437552


## Futures pricing

In [5]:
init_spot_price_oil_pb = 80

#define matrix complete matrix 
quad_var = np.matrix('0.000529 0.00003312 0.4471; 0.00003312 0.000144 0.0835; 0.44712 0.08352 576.0')
quad_var = quad_var * 0.0001

quad_var

matrix([[5.2900e-08, 3.3120e-09, 4.4710e-05],
        [3.3120e-09, 1.4400e-08, 8.3500e-06],
        [4.4712e-05, 8.3520e-06, 5.7600e-02]])

## Storage options